#Make sure to install Dependencies 

pip install tqdm

pip install asyncio

pip install aiofiles


In [20]:
import os
import re
import asyncio
import aiofiles
from tqdm import tqdm
from mboxCore import MBOX

In [21]:
# Specify the mbox filename
mobox_filename="mbox.full"
#mobox_filename="mbox_example"
mbox_dir= f'Resources/mbox/'
split_dir = 'Resources/Split/'
SPLIT_SIGNATURE='1.4.0.g6f2b'

In [22]:
if __name__ == "__main__":
    mbox1=MBOX(mbox_dir,split_dir,mobox_filename,SPLIT_SIGNATURE)
    asyncio.ensure_future(mbox1.mbox())

Spliting mbox file
Processing files


100%|██████████| 5/5 [00:00<00:00, 557.00it/s]


Input file: Resources/mbox/mbox.full
Output file is generated: Resources/mbox/mbox.full_reversed
